## An simple model to predict:
### 1. ability of each student
### 2. attributes of each problem
### 3. whether she/he will make a correct choice.

In [1]:
### import libraries
import numpy as np
import tensorflow as tf
import time
import xlrd, csv
import pandas as pd

In [24]:
### open the chemistry exam record 
workbook = xlrd.open_workbook('98_taiwan_5th.xlsx')
names = workbook.sheet_names()

xlsx = pd.ExcelFile('98_taiwan_5th.xlsx')
df1 = xlsx.parse(names[0], header=None)
df2 = xlsx.parse(names[1], header=None)
df3 = xlsx.parse(names[2], header=None)

In [14]:
df3 = df3.drop(df3.index[[0]])

In [20]:
df3.head()

,0,1,2,3,4,5,6,7,8,9,...,71,72,73,74,75,76,77,78,79,80
1,192130,0,0,1,0,1,0,0,0,0,...,1,0,0,0,1,0,1,0,0,0
2,191840,0,0,1,0,0,0,1,0,0,...,1,0,0,0,0,1,1,0,0,0
3,192094,0,0,1,0,1,0,0,0,0,...,1,0,1,0,0,0,1,0,0,0
4,192368,0,0,1,0,0,1,0,0,0,...,0,0,0,0,1,0,1,0,0,0
5,192018,0,0,0,1,1,0,0,0,0,...,0,1,0,1,0,0,0,0,0,1


In [26]:
df2

,0,1,2,3,4,5,6,7,8,9,...,68,69,70,71,72,73,74,75,76,77
0,192130,C,NaN,NaN,NaN,A,NaN,NaN,NaN,B,...,NaN,C,NaN,NaN,NaN,C,NaN,NaN,NaN,A
1,191840,C,NaN,NaN,NaN,C,NaN,NaN,NaN,B,...,NaN,C,NaN,NaN,NaN,D,NaN,NaN,NaN,A
2,192094,C,NaN,NaN,NaN,A,NaN,NaN,NaN,B,...,NaN,C,NaN,NaN,NaN,A,NaN,NaN,NaN,A
3,192368,C,NaN,NaN,NaN,B,NaN,NaN,NaN,B,...,NaN,A,NaN,NaN,NaN,C,NaN,NaN,NaN,A
4,192018,D,NaN,NaN,NaN,A,NaN,NaN,NaN,B,...,NaN,D,NaN,NaN,NaN,B,NaN,NaN,NaN,D
5,192159,C,NaN,NaN,NaN,B,NaN,NaN,NaN,B,...,NaN,C,NaN,NaN,NaN,C,NaN,NaN,NaN,A
6,192151,C,NaN,NaN,NaN,B,NaN,NaN,NaN,B,...,NaN,C,NaN,NaN,NaN,C,NaN,NaN,NaN,A
7,192338,D,NaN,NaN,NaN,B,NaN,NaN,NaN,B,...,NaN,A,NaN,NaN,NaN,B,NaN,NaN,NaN,A
8,192330,C,NaN,NaN,NaN,A,NaN,NaN,NaN,B,...,NaN,C,NaN,NaN,NaN,C,NaN,NaN,NaN,A
9,191961,C,NaN,NaN,NaN,A,NaN,NaN,NaN,B,...,NaN,A,NaN,NaN,NaN,C,NaN,NaN,NaN,A


In [28]:
df2 = df2.drop(df2.index[list(range(38,48))  ])

In [30]:
df2_ = df2.dropna(axis = 1)

In [31]:
df2_

,0,1,5,9,13,17,21,25,29,33,...,41,45,49,53,57,61,65,69,73,77
0,192130,C,A,B,B,D,D,C,A,B,...,A,C,D,A,C,A,B,C,C,A
1,191840,C,C,B,B,B,D,C,A,B,...,D,B,D,D,B,B,B,C,D,A
2,192094,C,A,B,B,C,A,A,A,B,...,D,B,D,C,C,C,B,C,A,A
3,192368,C,B,B,B,D,B,C,A,B,...,A,D,C,C,C,A,B,A,C,A
4,192018,D,A,B,D,B,C,D,D,C,...,B,D,A,D,A,A,C,D,B,D
5,192159,C,B,B,B,D,D,C,A,B,...,A,B,D,C,A,A,B,C,C,A
6,192151,C,B,B,B,D,D,C,A,B,...,C,A,D,D,C,B,B,C,C,A
7,192338,D,B,B,A,D,C,B,A,B,...,A,D,D,A,A,B,B,A,B,A
8,192330,C,A,B,B,C,D,C,A,B,...,C,,C,C,C,A,B,C,C,A
9,191961,C,A,B,B,C,B,C,A,B,...,D,B,D,A,A,A,B,A,C,A


In [32]:
df2_.columns = range(21)

In [33]:
df2_.loc[37][1:21]

1     C
2     A
3     B
4     B
5     D
6     D
7     C
8     A
9     B
10    D
11    A
12    C
13    D
14    D
15    C
16    A
17    B
18    C
19    C
20    A
Name: 37, dtype: object

In [32]:
### process the data 
df2_ = df2.dropna(axis = 1)
df2_.columns = range(21)

scores = []
for i_ in range(0,56):
    score = ((df2_.loc[i_][1:21]==df2_.loc[56][1:21]))
    scores.append(score)

scores = (np.array(scores)).astype(np.float)

In [33]:
### "scores" indicates how each student scores on each problem
###  (3 students, 10 problems)
pd.DataFrame(scores)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0
1,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
2,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0
3,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
4,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0
5,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
6,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0
7,0.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0
8,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
9,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0


In [34]:
'''
these parameters means: 
number of students
number of problems
number of latent factors
ratio of missing values and complete data
number of missing values
hyper parameters 1, to restrict the sum of elements in each problem to be one
hyper parameters 2, to restrict elements in each problem to be less than one  
'''
n_student = 56
n_prob = 20
n_latent = 6
ratio = 0.15
n_zeros = int(n_student*n_prob* ratio)
lamb1 = 0.1
lamb2 = 5.  

In [35]:
# proj_mat defines positions to drop a ratio of data randomly
# (optional)
proj_mat = np.ones(n_student*n_prob)
proj_mat[:n_zeros] = 0
np.random.shuffle( proj_mat )
proj_mat = proj_mat.reshape([n_student, n_prob])
test_mat = np.ones([n_student, n_prob]) - proj_mat

In [36]:
proj_mat.shape

(56, 20)

In [13]:
#test_mat

In [1]:
import numpy as np
import tensorflow as tf
import time
import xlrd, csv
import pandas as pd
workbook = xlrd.open_workbook('98_taiwan_5th.xlsx')
names = workbook.sheet_names()

xlsx = pd.ExcelFile('98_taiwan_5th.xlsx')
df1 = xlsx.parse(names[0], header=None)
df2 = xlsx.parse(names[1], header=None)
df3 = xlsx.parse(names[2], header=None)
df3 = df3.drop(df3.index[[0]])

df2 = df2.drop(df2.index[list(range(38,48))  ])  # for 98_taiwan_5th.xlsx
df2_ = df2.dropna(axis = 1)
df2_.columns = range(21)
df2_ = df2.dropna(axis = 1)
df2_.columns = range(21)

scores = []
for i_ in range(0,37):
    score = ((df2_.loc[i_][1:21]==df2_.loc[37][1:21]))
    scores.append(score)

scores = (np.array(scores)).astype(np.float)
n_student = 37
n_prob = 20
n_latent = 6
ratio = 0.1
n_zeros = int(n_student*n_prob* ratio)
lamb1 = 0.0001
lamb2 = 5.  
proj_mat = np.ones(n_student*n_prob)
proj_mat[:n_zeros] = 0
np.random.shuffle( proj_mat )
proj_mat = proj_mat.reshape([n_student, n_prob])
test_mat = np.ones([n_student, n_prob]) - proj_mat

### Use tensorflow(TF) to construct model:

In [2]:
sess = tf.Session()

In [3]:
class recomend:
    def __init__(self, n_student, n_prob, n_latent, ratio, lamb1, lamb2):
        self.ones = tf.constant(np.ones([n_student,n_prob]),
                                dtype=tf.float32)
        self.n_student = n_student
        self.n_prob = n_prob
        #self.lamb = lamb
        self.scores = tf.placeholder(dtype = tf.float32 ,
                                     shape = [n_student, n_prob])
        self.proj_ = tf.placeholder(dtype = tf.float32 ,
                                     shape = [n_student, n_prob])
        self.s_ = tf.get_variable("student",
                                     shape=[n_student, n_latent])
        self.s_bias = tf.get_variable("student_bias",
                                     shape=[n_student, 1])
        self.p_ = tf.get_variable("problem", shape=[n_prob, n_latent])
        self.p_bias = tf.get_variable("problem_bias", shape=[n_prob, 1])
        
#        self.predict =  tf.sigmoid(
#            tf.matmul(self.s_, tf.transpose(self.p_) ) + 
#            tf.tile(self.s_bias, [1, n_prob]) +
#            tf.transpose(tf.tile(self.p_bias, [1, n_student]))
#                                  )

        self.predict =  tf.sigmoid(
            tf.matmul(self.s_, tf.transpose(self.p_) ) + 
            tf.tile(self.s_bias, [1, n_prob])    )
        
#        self.predict =  tf.sigmoid(
#            tf.matmul(self.s_, tf.transpose(self.p_) )     )


#        self.loss_0 = tf.reduce_sum(tf.mul(-( 
#                                 self.scores*tf.log((3./4.)*self.predict + (1./4.)*self.ones  ) 
#                   + 3*(self.ones-self.scores)*tf.log((1./4.)*self.ones - (1./4.)* self.predict  +1e-6)   
#                                        ),self.proj_  ))/(n_student*n_prob* (1-ratio) )

#        self.loss_0 = tf.reduce_sum(tf.mul(-(
#          tf.mul(  self.scores, tf.log(self.predict + 1e-7  ) )    
#      + 3*tf.mul( (self.ones-self.scores),tf.log((1./3.)*(self.ones - self.predict)  +1e-7) )   
#                                             ),self.proj_  ))/(n_student*n_prob* (1-ratio) )
    
        self.loss_0 = tf.reduce_sum(tf.mul(-(
          tf.mul(  self.scores, tf.log(self.predict + 1e-6  ) )    
        + tf.mul( (self.ones-self.scores),tf.log(self.ones - self.predict  +1e-6) )   
                                             ),self.proj_  ))/(n_student*n_prob* (1-ratio) )
    
    
        self.loss_1 = lamb1 * (tf.reduce_sum(tf.square(self.p_) ) 
                              + tf.reduce_sum(tf.square(self.s_) )  )  #/((n_student+n_prob)*n_latent)
#        loss_0 = tf.reduce_sum(tf.square(self.predict-self.scores) )/(n_student*n_prob* (1-ratio) )
#        loss_1 = lamb1 * tf.reduce_sum(tf.square(tf.reduce_sum(self.p_, reduction_indices = 1) - self.ones) )
#        loss_2 = lamb2 * tf.reduce_sum(tf.pow(self.p_, 8) )/(n_student*n_prob* (1-ratio) )
#        self.loss = loss_0 + loss_1 + loss_2
        self.loss = self.loss_0 + self.loss_1

In [4]:
model = recomend(n_student, n_prob, n_latent, ratio, lamb1, lamb2)

In [5]:
#train_op = tf.train.GradientDescentOptimizer(1e-3).minimize(model.loss)
train_op = tf.train.AdamOptimizer(1e-4).minimize(model.loss)

In [6]:
sess.run(tf.initialize_all_variables() )

In [7]:
## start training:
delta = 10
loss_old = 1e3
steps = 0
t0 = 0
while loss_old > 23e-4:
    sess.run(train_op, feed_dict = {model.scores:scores,
                                    model.proj_: proj_mat })
    loss_0 = sess.run(model.loss_0, feed_dict = {model.scores:scores,
                                    model.proj_: proj_mat })
    loss_1 = sess.run(model.loss_1, feed_dict = {model.scores:scores,
                                    model.proj_: proj_mat })
#    loss_new = sess.run(model.loss, feed_dict = {model.scores:scores,
#                                    model.proj_: proj_mat })
    loss_new = loss_0 + loss_1
#    delta = loss_old - loss_new
    loss_old = loss_new
    steps +=1
    if steps%5000 ==0:
        t1 = time.time() - t0
        print(steps, loss_0, loss_1, t1)
        t0 = time.time()

(5000, 0.36948594, 0.010420809, 1521393876.470282)
(10000, 0.19774051, 0.024191806, 7.1697399616241455)
(15000, 0.097081408, 0.041029077, 7.162209987640381)
(20000, 0.050006986, 0.057198022, 7.185176849365234)
(25000, 0.033152658, 0.064866669, 7.136077165603638)
(30000, 0.028650232, 0.065372713, 7.1337058544158936)
(35000, 0.027454633, 0.064060055, 7.172659873962402)
(40000, 0.02648351, 0.063229464, 7.152651071548462)
(45000, 0.02550279, 0.062848806, 7.136449098587036)
(50000, 0.024960335, 0.06216383, 7.207020044326782)
(55000, 0.0241274, 0.062093027, 7.151232957839966)


KeyboardInterrupt: 

In [8]:
## final loss  and   delta
print(loss_new, delta)

(0.085881285, 10)


### Let's check the results:

In [20]:
#print(sess.run(model.s_))

In [9]:
model_s_result = sess.run(model.s_)
np.max(model_s_result)

2.8243093

In [45]:
#print(sess.run(model.p_))

In [47]:
#print(sess.run(model.s_bias) )

In [48]:
#print(sess.run(model.p_bias))

### counting wrong predicts

In [10]:
predict = sess.run(model.predict)
n_wrong = int(np.sum(np.square((np.sign(predict-0.25)+1)/2  - scores)*test_mat))
print(str(n_wrong)+' wrong predictions within '+ str(n_zeros) +' test samples' )

27 wrong predictions within 74 test samples


In [11]:
(74-27)/74.

0.6351351351351351

In [ ]:
### model 1:  for gary106A
'''
n_student = 56
n_prob = 20
n_latent = 6
ratio = 0.15
n_zeros = int(n_student*n_prob* ratio)
lamb1 = 1e-5

test 1:
loss = 
36 wrong predictions within 168 test samples
accuracy = 78.6%
'''

In [ ]:
### model 2: for gary106A
'''
n_student = 56
n_prob = 20
n_latent = 6
ratio = 0.15
n_zeros = int(n_student*n_prob* ratio)
lamb1 = 1e-4

test 1:
loss = 
30 wrong predictions within 168 test samples
accuracy = 82.1%

test 2:
loss = 0.085831843
22 wrong predictions within 168 test samples
accuracy = 86.9%

test 3:
loss = 0.075528845
26 wrong predictions within 168 test samples
accuracy = 84.5%
'''

In [ ]:
### model 2: for gary106B
'''
n_student = 56
n_prob = 20
n_latent = 6
ratio = 0.15
n_zeros = int(n_student*n_prob* ratio)
lamb1 = 1e-4

test 1:
loss = 0.10784461
45 wrong predictions within 165 test samples
accuracy = 72.7%

test 2:
loss = 0.10517757
43 wrong predictions within 165 test samples
accuracy = 73.9%


In [ ]:
### model 3: for gary106B
'''
n_student = 56
n_prob = 20
n_latent = 6
ratio = 0.1
n_zeros = int(n_student*n_prob* ratio)
lamb1 = 1e-4

test 1:
loss = 0.110769
28 wrong predictions within 110 test samples
accuracy = 74.5%

test 2:
loss = 0.11406308
22 wrong predictions within 110 test samples
accuracy = 74.4%


In [ ]:
### model 4: for gary106B
'''
n_student = 37
n_prob = 20
n_latent = 6
ratio = 0.1
n_zeros = int(n_student*n_prob* ratio)
lamb1 = 1e-4

test 1:
loss = 0.086653635
15 wrong predictions within 74 test samples
accuracy = 79.7%

test 2:
loss = 0.085881285
27 wrong predictions within 74 test samples
accuracy = 63.5%

'''

### All predict results 

In [13]:
print(predict)

[[  9.96689141e-01   1.00000000e+00   8.94945323e-01   9.95385110e-01
    1.50203705e-02   9.96655941e-01   9.93414402e-01   1.00000000e+00
    9.99404907e-01   9.99826491e-01   2.35673389e-03   3.62328766e-03
    1.00000000e+00   1.00000000e+00   9.97339189e-01   9.93513405e-01
    1.09186508e-02   9.99991179e-01   9.99994159e-01   9.98223722e-01]
 [  9.93370056e-01   9.99989867e-01   9.99999642e-01   9.96121109e-01
    9.94469404e-01   9.99999881e-01   9.99999523e-01   1.00000000e+00
    9.97487664e-01   1.04595963e-02   1.30576165e-02   9.99993205e-01
    9.95268047e-01   1.00000000e+00   9.99995828e-01   9.99999642e-01
    9.99873400e-01   1.00000000e+00   1.00000000e+00   9.99915123e-01]
 [  9.99829412e-01   9.99997616e-01   9.99918699e-01   1.00000000e+00
    9.92370248e-01   5.27903996e-03   9.99958277e-01   9.99993801e-01
    1.00000000e+00   9.99963284e-01   9.99994040e-01   5.52533334e-03
    9.99022365e-01   9.98921275e-01   9.99999762e-01   5.71044302e-03
    9.99422073e-01

In [14]:
print(scores)

[[ 1.  1.  1.  1.  0.  1.  1.  0.  1.  1.  1.  0.  1.  1.  1.  1.  0.  1.
   1.  1.]
 [ 1.  1.  1.  1.  1.  1.  1.  1.  1.  0.  0.  1.  1.  1.  1.  1.  1.  1.
   1.  1.]
 [ 1.  1.  1.  1.  1.  0.  1.  1.  1.  1.  1.  0.  1.  1.  1.  0.  1.  1.
   1.  1.]
 [ 1.  1.  1.  1.  1.  1.  1.  1.  1.  0.  0.  1.  1.  1.  1.  1.  1.  1.
   1.  1.]
 [ 1.  1.  0.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  0.  1.  1.
   1.  1.]
 [ 1.  1.  1.  0.  1.  1.  1.  1.  1.  0.  1.  1.  1.  1.  1.  1.  1.  1.
   1.  1.]
 [ 1.  1.  1.  1.  1.  1.  1.  1.  1.  0.  0.  1.  1.  0.  1.  1.  1.  1.
   0.  1.]
 [ 0.  0.  1.  1.  0.  1.  1.  0.  1.  1.  1.  0.  1.  0.  1.  0.  1.  0.
   1.  1.]
 [ 1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
   1.  1.]
 [ 1.  1.  1.  0.  1.  1.  1.  1.  1.  0.  1.  1.  1.  1.  1.  1.  1.  1.
   1.  1.]
 [ 0.  0.  1.  0.  0.  1.  1.  1.  0.  0.  0.  1.  0.  1.  1.  0.  1.  1.
   1.  0.]
 [ 1.  1.  1.  0.  1.  1.  1.  0.  1.  1.  1.  1.  1.  1.  1.  1.

In [15]:
ans_p = sess.run(model.p_, feed_dict = {model.scores:scores, model.proj_: proj_mat })
ans_s = sess.run(model.s_, feed_dict = {model.scores:scores, model.proj_: proj_mat })

In [16]:
print(np.max(ans_p))
print(np.max(ans_s))
print(np.max(p_mat))
print(np.max(s_mat))

4.30879
3.82516


NameError: name 'p_mat' is not defined

### The matrix "scores" corresponds to the actual scores students get,  each row corresponds to one student

In [19]:
scores

array([[ 52.67019706,  77.35301135,  59.15939095, ...,  59.93032837,
          0.        ,   0.        ],
       [ 47.74065807,  37.63227162,  35.40073239, ...,  54.0015399 ,
         55.28178138,  42.90616783],
       [ 46.38629337,  41.00573029,   0.        , ...,  40.05709955,
         45.1835761 ,  32.13017566],
       ..., 
       [ 32.62758583,  37.35290522,   0.        , ...,  41.88571342,
         34.00440983,  45.5966174 ],
       [ 44.59803368,  47.51215155,  42.46353872, ...,  35.88972598,
          0.        ,  19.03881936],
       [ 64.49604241,  41.26206108,  55.01551282, ...,  49.74749856,
         66.05305759,  57.30576901]])

### The matrix "predict" corresponds to the scores predicted by the model

In [20]:
predict = sess.run(model.predict, feed_dict = {model.scores:scores, model.proj_: proj_mat })
predict 

array([[ 50.31025696,  51.0971756 ,  50.85494995, ...,  49.55835342,
          0.        ,   0.        ],
       [ 42.00656891,  42.65892029,  42.45944595, ...,  41.37759399,
         41.76087952,  41.22695923],
       [ 51.43185425,  52.22583389,   0.        , ...,  50.65138626,
         51.12961197,  50.46997452],
       ..., 
       [ 31.10415268,  31.58787346,   0.        , ...,  30.63759232,
         30.92169189,  30.52473068],
       [ 41.2207756 ,  41.86265564,  41.66644669, ...,  40.59862518,
          0.        ,  40.44944   ],
       [ 60.9866066 ,  61.92451477,  61.64091492, ...,  60.0588913 ,
         60.62866211,  59.8453331 ]], dtype=float32)

In [21]:
p_mat[0]

array([ 0.2280732 ,  0.17640353,  0.06800562,  0.20650069,  0.32101697])

In [22]:
ans_p[0]

array([ 1.02335465,  1.11626089, -1.13355446,  1.12117815, -1.12711513], dtype=float32)